In [10]:
# Upset Constraints

from re import T
import pandas as pd
import requests
import io
from math import *
from gurobipy import *
import matplotlib.pyplot as plt

# Pull in initial CSV file from Github and transform it into a Dataframe

url = "https://raw.githubusercontent.com/camjm21/4133project/main/March%20Madness.csv"
download = requests.get(url).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.set_index('TEAM', inplace=True)

#Pull in actual 2019 bracket results
url2 = "https://raw.githubusercontent.com/camjm21/4133project/main/2019ActualBracket.csv"
download2 = requests.get(url2).content
origninalBracket = pd.read_csv(io.StringIO(download2.decode('utf-8')))
origninalBracket.rename(columns = {"Unnamed: 0": "TEAM"}, inplace = True)
origninalBracket.set_index('TEAM', inplace = True)


# Create dictionary for round one matchups, every team will be a key, with their opponent being the value
teams = list(df.index)
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 2 == 0:
        values[i+1] = teams[i]
    else:
        values[i-1] = teams[i]

firstround = dict(zip(teams, values))

# Create dataframe to put results in
results = pd.DataFrame(index=teams,columns=['First Round','Second Round', 'Third Round', 'Fourth Round', 'Fifth Round', 'Sixth Round'])

# Put first round probabilities in
results['First Round'] = [df.loc[i, firstround[i]] for i in teams]

# Create dictionary for round two matchups, same format as round one, but the values are lists of the two possible opponents they could face in round two
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 4 < 2:
        values[i] = [teams[4*floor(i/4)+j] for j in range(2,4)]
    else:
        values[i] = [teams[4*ceil(i/4)-j] for j in range(4,2, -1)]

secondround = dict(zip(teams, values))

# Put second round probabilities in
results['Second Round'] = [sum(df.loc[i, j]*results.loc[j, 'First Round']*results.loc[i, 'First Round'] for j in secondround[i]) for i in teams ]

# Create dictionary for round three matchups, same format as round two
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 8 < 4:
        values[i] = [teams[8*floor(i/8)+j] for j in range(4,8)]
    else:
        values[i] = [teams[8*ceil(i/8)-j] for j in range(8,4, -1)]


thirdround = dict(zip(teams, values))

# Put third round probabilities in
results['Third Round'] = [sum(df.loc[i, j]*results.loc[j, 'Second Round']*results.loc[i, 'Second Round'] for j in thirdround[i]) for i in teams]

# Create dictionary for round four matchups, same format as round three
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 16 < 8:
        values[i] = [teams[16*floor(i/16)+j] for j in range(8,16)]
    else:
        values[i] = [teams[16*ceil(i/16)-j] for j in range(16,8, -1)]
        
fourthround = dict(zip(teams, values))

# Put fourth round probabilities in
results['Fourth Round'] = [sum(df.loc[i, j]*results.loc[j, 'Third Round']*results.loc[i, 'Third Round'] for j in fourthround[i]) for i in teams]

# Create dictionary for round five matchups, same format as round four
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 32 < 16:
        values[i] = [teams[32*floor(i/32)+j] for j in range(16,32)]
    else:
        values[i] = [teams[32*ceil(i/32)-j] for j in range(32,16, -1)]

fifthround = dict(zip(teams, values))

# Put fifth round probabilities in
results['Fifth Round'] = [sum(df.loc[i, j]*results.loc[j, 'Fourth Round']*results.loc[i, 'Fourth Round'] for j in fifthround[i]) for i in teams]

# Create dictionary for round six matchups, same format as round five
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 64 < 32:
        values[i] = [teams[64*floor(i/64)+j] for j in range(32,64)]
    else:
        values[i] = [teams[64*ceil(i/64)-j] for j in range(64,32, -1)]


sixthround = dict(zip(teams, values))

# Put sixth round probabilities in
results['Sixth Round'] = [sum(df.loc[i, j]*results.loc[j, 'Fifth Round']*results.loc[i, 'Fifth Round'] for j in sixthround[i]) for i in teams]



# Create model and Indices
m = Model("March Madness")
rounds = results.columns.tolist()
weights = {'First Round' : 1, 'Second Round' : 2, 'Third Round' :4, 'Fourth Round' : 8, 'Fifth Round' : 16, 'Sixth Round' : 32}

# Create decision variables
x = m.addVars(teams, rounds, vtype = GRB.BINARY, name = 'x')

# Create objective function
m.setObjective(quicksum(weights[k] * x[i, k] * results.loc[i, k] for i in teams for k in rounds), GRB.MAXIMIZE)


# Create constraints

# Round 1 Constraints
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Duke', 'N Dakota St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['VA Commonwealth', 'UCF']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Mississippi St', 'Liberty']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Virginia Tech', 'St Louis']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Maryland', 'Belmont']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['LSU', 'Yale']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Louisville', 'Minnesota']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Michigan St', 'Bradley']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Gonzaga', 'F Dickinson']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Syracuse', 'Baylor']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Marquette', 'Murray St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Florida St', 'Vermont']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Buffalo', 'Arizona St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Texas Tech', 'N Kentucky']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Nevada', 'Florida']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Michigan', 'Montana']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Virginia', 'Gardner Webb']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Mississippi', 'Oklahoma']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Wisconsin', 'Oregon']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Kansas St', 'UC Irvine']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Villanova', "St Mary's CA"]) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Purdue', 'Old Dominion']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Cincinnati', 'Iowa']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Tennessee', 'Colgate']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['North Carolina', 'Iona']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Utah St', 'Washington']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Auburn', 'New Mexico St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Kansas', 'Northeastern']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Iowa St', 'Ohio St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Houston', 'Georgia St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Wofford', 'Seton Hall']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Kentucky', 'Abilene Chr']) == 1)

# Round 2 Constraints
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Duke', 'N Dakota St', 'VA Commonwealth', 'UCF']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Mississippi St', 'Liberty', 'Virginia Tech', 'St Louis']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Maryland', 'Belmont', 'LSU', 'Yale']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Louisville', 'Minnesota', 'Michigan St', 'Bradley']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Gonzaga', 'F Dickinson', 'Syracuse', 'Baylor']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Marquette', 'Murray St', 'Florida St', 'Vermont']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Buffalo', 'Arizona St', 'Texas Tech', 'N Kentucky']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Nevada', 'Florida', 'Michigan', 'Montana']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Virginia', 'Gardner Webb', 'Mississippi', 'Oklahoma']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Wisconsin', 'Oregon', 'Kansas St', 'UC Irvine']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Villanova', "St Mary's CA", 'Purdue', 'Old Dominion']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Cincinnati', 'Iowa', 'Tennessee', 'Colgate']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['North Carolina', 'Iona', 'Utah St', 'Washington']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Auburn', 'New Mexico St', 'Kansas', 'Northeastern']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Iowa St', 'Ohio St', 'Houston', 'Georgia St']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Wofford', 'Seton Hall', 'Kentucky', 'Abilene Chr']) == 1)

# Round 3 Constraints
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Duke', 'N Dakota St', 'VA Commonwealth', 'UCF', 'Mississippi St', 'Liberty', 'Virginia Tech', 'St Louis']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Maryland', 'Belmont', 'LSU', 'Yale', 'Louisville', 'Minnesota', 'Michigan St', 'Bradley']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Gonzaga', 'F Dickinson', 'Syracuse', 'Baylor', 'Marquette', 'Murray St', 'Florida St', 'Vermont']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Buffalo', 'Arizona St', 'Texas Tech', 'N Kentucky', 'Nevada', 'Florida', 'Michigan', 'Montana']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Virginia', 'Gardner Webb', 'Mississippi', 'Oklahoma', 'Wisconsin', 'Oregon', 'Kansas St', 'UC Irvine']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Villanova', "St Mary's CA", 'Purdue', 'Old Dominion', 'Cincinnati', 'Iowa', 'Tennessee', 'Colgate']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['North Carolina', 'Iona', 'Utah St', 'Washington', 'Auburn', 'New Mexico St', 'Kansas', 'Northeastern']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Iowa St', 'Ohio St', 'Houston', 'Georgia St', 'Wofford', 'Seton Hall', 'Kentucky', 'Abilene Chr']) == 1)

# Round 4 Constraints
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[0:16]) == 1)
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[16:32]) == 1)
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[32:48]) == 1)
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[48:64]) == 1)

# Round 5 Constraints
m.addConstr(quicksum(x[i, 'Fifth Round'] for i in teams[0:32]) == 1)
m.addConstr(quicksum(x[i, 'Fifth Round'] for i in teams[32:64]) == 1)

# Round 6 Constraints
m.addConstr(quicksum(x[i, 'Sixth Round'] for i in teams) == 1)

# Elimination Constraints
m.addConstrs(quicksum(x[i, k] for k in rounds[1:6]) <= 5*x[i, "First Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[2:6]) <= 4*x[i, "Second Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[3:6]) <= 3*x[i, "Third Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[4:6]) <= 2*x[i, "Fourth Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[5:6]) <= x[i, "Fifth Round"] for i in teams)

# Upset Constraints
m.addConstr(quicksum(x[i, 'First Round'] for i in teams[3::16]) >= 2)
m.addConstr(quicksum(x[i, 'First Round'] for i in teams[5::16]) >= 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in teams[9::16]) >= 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in teams[13::16]) >= 1)



# Optimize and record objective value
m.optimize()
objectiveScores = []
objectiveScores.append(m.ObjVal)

# Setup decision variable results from optimization in dataframe
resultsx = []
for v in m.getVars():
    #print("%s %g" % (v.varName, v.x))
    resultsx.append(v.x)
reshapedresultsx = [resultsx[i:i+6] for i in range(0, 384, 6)]
resultsDF = pd.DataFrame(reshapedresultsx, index = teams, columns=rounds)

#Compute bracket score based on how it matches 2019 bracket
bracketScores = []
bracketScore = 0
for i in teams:
    k = 0
    for j in rounds:
        if resultsDF.loc[i, j] == origninalBracket.loc[i, j] and origninalBracket.loc[i,j] == 1:
            bracketScore = bracketScore + 2**k
        k+=1
bracketScores.append(bracketScore)



# Repeat 9 more times 
for a in range(9):

    # Create new constraint forcing out a winning game to get a new bracket
    winningGames = [[i, j] for i in teams for j in rounds if resultsDF.loc[i, j] == 1]
    m.addConstr(quicksum(x[i, j] for [i, j] in winningGames) <= 62)

    # Optimize model with new constraint and record value
    m.optimize()
    objectiveScores.append(m.ObjVal)

    # Setup results from optimization in dataframe
    resultsx = []
    for v in m.getVars():
        #print("%s %g" % (v.varName, v.x))
        resultsx.append(v.x)
    reshapedresultsx = [resultsx[i:i+6] for i in range(0, 384, 6)]
    resultsDF = pd.DataFrame(reshapedresultsx, index = teams, columns=rounds)

    # Compute bracket score based on how it matches 2019 bracket
    bracketScore = 0
    for i in teams:
        k = 0
        for j in rounds:
            if resultsDF.loc[i, j] == origninalBracket.loc[i, j] and origninalBracket.loc[i,j] == 1:
                bracketScore = bracketScore + 2**k
            k+=1
    bracketScores.append(bracketScore)
    
# Print results
print(bracketScores)
print(objectiveScores)



Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 387 rows, 384 columns and 1680 nonzeros
Model fingerprint: 0xa388a19e
Variable types: 0 continuous, 384 integer (384 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [4e-11, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 37.5775295
Presolve removed 48 rows and 48 columns
Presolve time: 0.00s
Presolved: 339 rows, 336 columns, 1616 nonzeros
Variable types: 0 continuous, 336 integer (336 binary)

Root relaxation: objective 9.689139e+01, 50 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   96.89139    0    2   37.57753   96.89139   158%     -    0s
H    0     0                      96.7758683   96.8913

In [11]:
# Upper Seed Constraints

from re import T
import pandas as pd
import requests
import io
from math import *
from gurobipy import *
import matplotlib.pyplot as plt

# Pull in initial CSV file from Github and transform it into a Dataframe

url = "https://raw.githubusercontent.com/camjm21/4133project/main/March%20Madness.csv"
download = requests.get(url).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.set_index('TEAM', inplace=True)

#Pull in actual 2019 bracket results
url2 = "https://raw.githubusercontent.com/camjm21/4133project/main/2019ActualBracket.csv"
download2 = requests.get(url2).content
origninalBracket = pd.read_csv(io.StringIO(download2.decode('utf-8')))
origninalBracket.rename(columns = {"Unnamed: 0": "TEAM"}, inplace = True)
origninalBracket.set_index('TEAM', inplace = True)


# Create dictionary for round one matchups, every team will be a key, with their opponent being the value
teams = list(df.index)
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 2 == 0:
        values[i+1] = teams[i]
    else:
        values[i-1] = teams[i]

firstround = dict(zip(teams, values))

# Create dataframe to put results in
results = pd.DataFrame(index=teams,columns=['First Round','Second Round', 'Third Round', 'Fourth Round', 'Fifth Round', 'Sixth Round'])

# Put first round probabilities in
results['First Round'] = [df.loc[i, firstround[i]] for i in teams]

# Create dictionary for round two matchups, same format as round one, but the values are lists of the two possible opponents they could face in round two
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 4 < 2:
        values[i] = [teams[4*floor(i/4)+j] for j in range(2,4)]
    else:
        values[i] = [teams[4*ceil(i/4)-j] for j in range(4,2, -1)]

secondround = dict(zip(teams, values))

# Put second round probabilities in
results['Second Round'] = [sum(df.loc[i, j]*results.loc[j, 'First Round']*results.loc[i, 'First Round'] for j in secondround[i]) for i in teams ]

# Create dictionary for round three matchups, same format as round two
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 8 < 4:
        values[i] = [teams[8*floor(i/8)+j] for j in range(4,8)]
    else:
        values[i] = [teams[8*ceil(i/8)-j] for j in range(8,4, -1)]


thirdround = dict(zip(teams, values))

# Put third round probabilities in
results['Third Round'] = [sum(df.loc[i, j]*results.loc[j, 'Second Round']*results.loc[i, 'Second Round'] for j in thirdround[i]) for i in teams]

# Create dictionary for round four matchups, same format as round three
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 16 < 8:
        values[i] = [teams[16*floor(i/16)+j] for j in range(8,16)]
    else:
        values[i] = [teams[16*ceil(i/16)-j] for j in range(16,8, -1)]
        
fourthround = dict(zip(teams, values))

# Put fourth round probabilities in
results['Fourth Round'] = [sum(df.loc[i, j]*results.loc[j, 'Third Round']*results.loc[i, 'Third Round'] for j in fourthround[i]) for i in teams]

# Create dictionary for round five matchups, same format as round four
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 32 < 16:
        values[i] = [teams[32*floor(i/32)+j] for j in range(16,32)]
    else:
        values[i] = [teams[32*ceil(i/32)-j] for j in range(32,16, -1)]

fifthround = dict(zip(teams, values))

# Put fifth round probabilities in
results['Fifth Round'] = [sum(df.loc[i, j]*results.loc[j, 'Fourth Round']*results.loc[i, 'Fourth Round'] for j in fifthround[i]) for i in teams]

# Create dictionary for round six matchups, same format as round five
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 64 < 32:
        values[i] = [teams[64*floor(i/64)+j] for j in range(32,64)]
    else:
        values[i] = [teams[64*ceil(i/64)-j] for j in range(64,32, -1)]


sixthround = dict(zip(teams, values))

# Put sixth round probabilities in
results['Sixth Round'] = [sum(df.loc[i, j]*results.loc[j, 'Fifth Round']*results.loc[i, 'Fifth Round'] for j in sixthround[i]) for i in teams]



# Create model and Indices
m = Model("March Madness")
rounds = results.columns.tolist()
weights = {'First Round' : 1, 'Second Round' : 2, 'Third Round' :4, 'Fourth Round' : 8, 'Fifth Round' : 16, 'Sixth Round' : 32}

# Create decision variables
x = m.addVars(teams, rounds, vtype = GRB.BINARY, name = 'x')

# Create objective function
m.setObjective(quicksum(weights[k] * x[i, k] * results.loc[i, k] for i in teams for k in rounds), GRB.MAXIMIZE)


# Create constraints

# Round 1 Constraints
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Duke', 'N Dakota St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['VA Commonwealth', 'UCF']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Mississippi St', 'Liberty']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Virginia Tech', 'St Louis']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Maryland', 'Belmont']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['LSU', 'Yale']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Louisville', 'Minnesota']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Michigan St', 'Bradley']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Gonzaga', 'F Dickinson']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Syracuse', 'Baylor']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Marquette', 'Murray St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Florida St', 'Vermont']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Buffalo', 'Arizona St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Texas Tech', 'N Kentucky']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Nevada', 'Florida']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Michigan', 'Montana']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Virginia', 'Gardner Webb']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Mississippi', 'Oklahoma']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Wisconsin', 'Oregon']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Kansas St', 'UC Irvine']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Villanova', "St Mary's CA"]) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Purdue', 'Old Dominion']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Cincinnati', 'Iowa']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Tennessee', 'Colgate']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['North Carolina', 'Iona']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Utah St', 'Washington']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Auburn', 'New Mexico St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Kansas', 'Northeastern']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Iowa St', 'Ohio St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Houston', 'Georgia St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Wofford', 'Seton Hall']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Kentucky', 'Abilene Chr']) == 1)

# Round 2 Constraints
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Duke', 'N Dakota St', 'VA Commonwealth', 'UCF']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Mississippi St', 'Liberty', 'Virginia Tech', 'St Louis']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Maryland', 'Belmont', 'LSU', 'Yale']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Louisville', 'Minnesota', 'Michigan St', 'Bradley']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Gonzaga', 'F Dickinson', 'Syracuse', 'Baylor']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Marquette', 'Murray St', 'Florida St', 'Vermont']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Buffalo', 'Arizona St', 'Texas Tech', 'N Kentucky']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Nevada', 'Florida', 'Michigan', 'Montana']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Virginia', 'Gardner Webb', 'Mississippi', 'Oklahoma']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Wisconsin', 'Oregon', 'Kansas St', 'UC Irvine']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Villanova', "St Mary's CA", 'Purdue', 'Old Dominion']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Cincinnati', 'Iowa', 'Tennessee', 'Colgate']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['North Carolina', 'Iona', 'Utah St', 'Washington']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Auburn', 'New Mexico St', 'Kansas', 'Northeastern']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Iowa St', 'Ohio St', 'Houston', 'Georgia St']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Wofford', 'Seton Hall', 'Kentucky', 'Abilene Chr']) == 1)

# Round 3 Constraints
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Duke', 'N Dakota St', 'VA Commonwealth', 'UCF', 'Mississippi St', 'Liberty', 'Virginia Tech', 'St Louis']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Maryland', 'Belmont', 'LSU', 'Yale', 'Louisville', 'Minnesota', 'Michigan St', 'Bradley']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Gonzaga', 'F Dickinson', 'Syracuse', 'Baylor', 'Marquette', 'Murray St', 'Florida St', 'Vermont']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Buffalo', 'Arizona St', 'Texas Tech', 'N Kentucky', 'Nevada', 'Florida', 'Michigan', 'Montana']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Virginia', 'Gardner Webb', 'Mississippi', 'Oklahoma', 'Wisconsin', 'Oregon', 'Kansas St', 'UC Irvine']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Villanova', "St Mary's CA", 'Purdue', 'Old Dominion', 'Cincinnati', 'Iowa', 'Tennessee', 'Colgate']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['North Carolina', 'Iona', 'Utah St', 'Washington', 'Auburn', 'New Mexico St', 'Kansas', 'Northeastern']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Iowa St', 'Ohio St', 'Houston', 'Georgia St', 'Wofford', 'Seton Hall', 'Kentucky', 'Abilene Chr']) == 1)

# Round 4 Constraints
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[0:16]) == 1)
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[16:32]) == 1)
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[32:48]) == 1)
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[48:64]) == 1)

# Round 5 Constraints
m.addConstr(quicksum(x[i, 'Fifth Round'] for i in teams[0:32]) == 1)
m.addConstr(quicksum(x[i, 'Fifth Round'] for i in teams[32:64]) == 1)

# Round 6 Constraints
m.addConstr(quicksum(x[i, 'Sixth Round'] for i in teams) == 1)

# Elimination Constraints
m.addConstrs(quicksum(x[i, k] for k in rounds[1:6]) <= 5*x[i, "First Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[2:6]) <= 4*x[i, "Second Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[3:6]) <= 3*x[i, "Third Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[4:6]) <= 2*x[i, "Fourth Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[5:6]) <= x[i, "Fifth Round"] for i in teams)

# Upper Seed Constraints
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[0::16]) <= 3)
#m.addConstr(quicksum(x[i, 'Fourth Round'] for i in (teams[0::16] + teams[13::16])) >= 2)


# Optimize and record objective value
m.optimize()
objectiveScores = []
objectiveScores.append(m.ObjVal)

# Setup decision variable results from optimization in dataframe
resultsx = []
for v in m.getVars():
    #print("%s %g" % (v.varName, v.x))
    resultsx.append(v.x)
reshapedresultsx = [resultsx[i:i+6] for i in range(0, 384, 6)]
resultsDF = pd.DataFrame(reshapedresultsx, index = teams, columns=rounds)

#Compute bracket score based on how it matches 2019 bracket
bracketScores = []
bracketScore = 0
for i in teams:
    k = 0
    for j in rounds:
        if resultsDF.loc[i, j] == origninalBracket.loc[i, j] and origninalBracket.loc[i,j] == 1:
            bracketScore = bracketScore + 2**k
        k+=1
bracketScores.append(bracketScore)



# Repeat 9 more times 
for a in range(9):

    # Create new constraint forcing out a winning game to get a new bracket
    winningGames = [[i, j] for i in teams for j in rounds if resultsDF.loc[i, j] == 1]
    m.addConstr(quicksum(x[i, j] for [i, j] in winningGames) <= 62)

    # Optimize model with new constraint and record value
    m.optimize()
    objectiveScores.append(m.ObjVal)

    # Setup results from optimization in dataframe
    resultsx = []
    for v in m.getVars():
        #print("%s %g" % (v.varName, v.x))
        resultsx.append(v.x)
    reshapedresultsx = [resultsx[i:i+6] for i in range(0, 384, 6)]
    resultsDF = pd.DataFrame(reshapedresultsx, index = teams, columns=rounds)

    # Compute bracket score based on how it matches 2019 bracket
    bracketScore = 0
    for i in teams:
        k = 0
        for j in rounds:
            if resultsDF.loc[i, j] == origninalBracket.loc[i, j] and origninalBracket.loc[i,j] == 1:
                bracketScore = bracketScore + 2**k
            k+=1
    bracketScores.append(bracketScore)
    
# Print results
print(bracketScores)
print(objectiveScores)



Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 384 rows, 384 columns and 1668 nonzeros
Model fingerprint: 0xd2656b78
Variable types: 0 continuous, 384 integer (384 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [4e-11, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 37.5775295
Presolve removed 48 rows and 48 columns
Presolve time: 0.00s
Presolved: 336 rows, 336 columns, 1604 nonzeros
Variable types: 0 continuous, 336 integer (336 binary)

Root relaxation: objective 9.611787e+01, 41 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      96.1178699   96.11787  0.00%     -    0s

Explored 0 nodes (41 simplex iterations) in 0.01 seco

In [12]:
# Upset and Upper Seed Constraints

from re import T
import pandas as pd
import requests
import io
from math import *
from gurobipy import *
import matplotlib.pyplot as plt

# Pull in initial CSV file from Github and transform it into a Dataframe

url = "https://raw.githubusercontent.com/camjm21/4133project/main/March%20Madness.csv"
download = requests.get(url).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.set_index('TEAM', inplace=True)

#Pull in actual 2019 bracket results
url2 = "https://raw.githubusercontent.com/camjm21/4133project/main/2019ActualBracket.csv"
download2 = requests.get(url2).content
origninalBracket = pd.read_csv(io.StringIO(download2.decode('utf-8')))
origninalBracket.rename(columns = {"Unnamed: 0": "TEAM"}, inplace = True)
origninalBracket.set_index('TEAM', inplace = True)


# Create dictionary for round one matchups, every team will be a key, with their opponent being the value
teams = list(df.index)
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 2 == 0:
        values[i+1] = teams[i]
    else:
        values[i-1] = teams[i]

firstround = dict(zip(teams, values))

# Create dataframe to put results in
results = pd.DataFrame(index=teams,columns=['First Round','Second Round', 'Third Round', 'Fourth Round', 'Fifth Round', 'Sixth Round'])

# Put first round probabilities in
results['First Round'] = [df.loc[i, firstround[i]] for i in teams]

# Create dictionary for round two matchups, same format as round one, but the values are lists of the two possible opponents they could face in round two
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 4 < 2:
        values[i] = [teams[4*floor(i/4)+j] for j in range(2,4)]
    else:
        values[i] = [teams[4*ceil(i/4)-j] for j in range(4,2, -1)]

secondround = dict(zip(teams, values))

# Put second round probabilities in
results['Second Round'] = [sum(df.loc[i, j]*results.loc[j, 'First Round']*results.loc[i, 'First Round'] for j in secondround[i]) for i in teams ]

# Create dictionary for round three matchups, same format as round two
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 8 < 4:
        values[i] = [teams[8*floor(i/8)+j] for j in range(4,8)]
    else:
        values[i] = [teams[8*ceil(i/8)-j] for j in range(8,4, -1)]


thirdround = dict(zip(teams, values))

# Put third round probabilities in
results['Third Round'] = [sum(df.loc[i, j]*results.loc[j, 'Second Round']*results.loc[i, 'Second Round'] for j in thirdround[i]) for i in teams]

# Create dictionary for round four matchups, same format as round three
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 16 < 8:
        values[i] = [teams[16*floor(i/16)+j] for j in range(8,16)]
    else:
        values[i] = [teams[16*ceil(i/16)-j] for j in range(16,8, -1)]
        
fourthround = dict(zip(teams, values))

# Put fourth round probabilities in
results['Fourth Round'] = [sum(df.loc[i, j]*results.loc[j, 'Third Round']*results.loc[i, 'Third Round'] for j in fourthround[i]) for i in teams]

# Create dictionary for round five matchups, same format as round four
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 32 < 16:
        values[i] = [teams[32*floor(i/32)+j] for j in range(16,32)]
    else:
        values[i] = [teams[32*ceil(i/32)-j] for j in range(32,16, -1)]

fifthround = dict(zip(teams, values))

# Put fifth round probabilities in
results['Fifth Round'] = [sum(df.loc[i, j]*results.loc[j, 'Fourth Round']*results.loc[i, 'Fourth Round'] for j in fifthround[i]) for i in teams]

# Create dictionary for round six matchups, same format as round five
values = [0] * len(teams)
for i in range(len(teams)):
    if i % 64 < 32:
        values[i] = [teams[64*floor(i/64)+j] for j in range(32,64)]
    else:
        values[i] = [teams[64*ceil(i/64)-j] for j in range(64,32, -1)]


sixthround = dict(zip(teams, values))

# Put sixth round probabilities in
results['Sixth Round'] = [sum(df.loc[i, j]*results.loc[j, 'Fifth Round']*results.loc[i, 'Fifth Round'] for j in sixthround[i]) for i in teams]



# Create model and Indices
m = Model("March Madness")
rounds = results.columns.tolist()
weights = {'First Round' : 1, 'Second Round' : 2, 'Third Round' :4, 'Fourth Round' : 8, 'Fifth Round' : 16, 'Sixth Round' : 32}

# Create decision variables
x = m.addVars(teams, rounds, vtype = GRB.BINARY, name = 'x')

# Create objective function
m.setObjective(quicksum(weights[k] * x[i, k] * results.loc[i, k] for i in teams for k in rounds), GRB.MAXIMIZE)


# Create constraints

# Round 1 Constraints
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Duke', 'N Dakota St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['VA Commonwealth', 'UCF']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Mississippi St', 'Liberty']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Virginia Tech', 'St Louis']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Maryland', 'Belmont']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['LSU', 'Yale']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Louisville', 'Minnesota']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Michigan St', 'Bradley']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Gonzaga', 'F Dickinson']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Syracuse', 'Baylor']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Marquette', 'Murray St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Florida St', 'Vermont']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Buffalo', 'Arizona St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Texas Tech', 'N Kentucky']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Nevada', 'Florida']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Michigan', 'Montana']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Virginia', 'Gardner Webb']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Mississippi', 'Oklahoma']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Wisconsin', 'Oregon']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Kansas St', 'UC Irvine']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Villanova', "St Mary's CA"]) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Purdue', 'Old Dominion']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Cincinnati', 'Iowa']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Tennessee', 'Colgate']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['North Carolina', 'Iona']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Utah St', 'Washington']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Auburn', 'New Mexico St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Kansas', 'Northeastern']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Iowa St', 'Ohio St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Houston', 'Georgia St']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Wofford', 'Seton Hall']) == 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in ['Kentucky', 'Abilene Chr']) == 1)

# Round 2 Constraints
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Duke', 'N Dakota St', 'VA Commonwealth', 'UCF']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Mississippi St', 'Liberty', 'Virginia Tech', 'St Louis']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Maryland', 'Belmont', 'LSU', 'Yale']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Louisville', 'Minnesota', 'Michigan St', 'Bradley']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Gonzaga', 'F Dickinson', 'Syracuse', 'Baylor']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Marquette', 'Murray St', 'Florida St', 'Vermont']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Buffalo', 'Arizona St', 'Texas Tech', 'N Kentucky']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Nevada', 'Florida', 'Michigan', 'Montana']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Virginia', 'Gardner Webb', 'Mississippi', 'Oklahoma']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Wisconsin', 'Oregon', 'Kansas St', 'UC Irvine']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Villanova', "St Mary's CA", 'Purdue', 'Old Dominion']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Cincinnati', 'Iowa', 'Tennessee', 'Colgate']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['North Carolina', 'Iona', 'Utah St', 'Washington']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Auburn', 'New Mexico St', 'Kansas', 'Northeastern']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Iowa St', 'Ohio St', 'Houston', 'Georgia St']) == 1)
m.addConstr(quicksum(x[i, 'Second Round'] for i in ['Wofford', 'Seton Hall', 'Kentucky', 'Abilene Chr']) == 1)

# Round 3 Constraints
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Duke', 'N Dakota St', 'VA Commonwealth', 'UCF', 'Mississippi St', 'Liberty', 'Virginia Tech', 'St Louis']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Maryland', 'Belmont', 'LSU', 'Yale', 'Louisville', 'Minnesota', 'Michigan St', 'Bradley']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Gonzaga', 'F Dickinson', 'Syracuse', 'Baylor', 'Marquette', 'Murray St', 'Florida St', 'Vermont']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Buffalo', 'Arizona St', 'Texas Tech', 'N Kentucky', 'Nevada', 'Florida', 'Michigan', 'Montana']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Virginia', 'Gardner Webb', 'Mississippi', 'Oklahoma', 'Wisconsin', 'Oregon', 'Kansas St', 'UC Irvine']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Villanova', "St Mary's CA", 'Purdue', 'Old Dominion', 'Cincinnati', 'Iowa', 'Tennessee', 'Colgate']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['North Carolina', 'Iona', 'Utah St', 'Washington', 'Auburn', 'New Mexico St', 'Kansas', 'Northeastern']) == 1)
m.addConstr(quicksum(x[i, 'Third Round'] for i in ['Iowa St', 'Ohio St', 'Houston', 'Georgia St', 'Wofford', 'Seton Hall', 'Kentucky', 'Abilene Chr']) == 1)

# Round 4 Constraints
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[0:16]) == 1)
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[16:32]) == 1)
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[32:48]) == 1)
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[48:64]) == 1)

# Round 5 Constraints
m.addConstr(quicksum(x[i, 'Fifth Round'] for i in teams[0:32]) == 1)
m.addConstr(quicksum(x[i, 'Fifth Round'] for i in teams[32:64]) == 1)

# Round 6 Constraints
m.addConstr(quicksum(x[i, 'Sixth Round'] for i in teams) == 1)

# Elimination Constraints
m.addConstrs(quicksum(x[i, k] for k in rounds[1:6]) <= 5*x[i, "First Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[2:6]) <= 4*x[i, "Second Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[3:6]) <= 3*x[i, "Third Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[4:6]) <= 2*x[i, "Fourth Round"] for i in teams)
m.addConstrs(quicksum(x[i, k] for k in rounds[5:6]) <= x[i, "Fifth Round"] for i in teams)

# Upset Constraints
m.addConstr(quicksum(x[i, 'First Round'] for i in teams[3::16]) >= 2)
m.addConstr(quicksum(x[i, 'First Round'] for i in teams[5::16]) >= 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in teams[9::16]) >= 1)
m.addConstr(quicksum(x[i, 'First Round'] for i in teams[13::16]) >= 1)

# Upper Seed Constraints
m.addConstr(quicksum(x[i, 'Fourth Round'] for i in teams[0::16]) <= 3)


# Optimize and record objective value
m.optimize()
objectiveScores = []
objectiveScores.append(m.ObjVal)

# Setup decision variable results from optimization in dataframe
resultsx = []
for v in m.getVars():
    #print("%s %g" % (v.varName, v.x))
    resultsx.append(v.x)
reshapedresultsx = [resultsx[i:i+6] for i in range(0, 384, 6)]
resultsDF = pd.DataFrame(reshapedresultsx, index = teams, columns=rounds)

#Compute bracket score based on how it matches 2019 bracket
bracketScores = []
bracketScore = 0
for i in teams:
    k = 0
    for j in rounds:
        if resultsDF.loc[i, j] == origninalBracket.loc[i, j] and origninalBracket.loc[i,j] == 1:
            bracketScore = bracketScore + 2**k
        k+=1
bracketScores.append(bracketScore)



# Repeat 9 more times 
for a in range(9):

    # Create new constraint forcing out a winning game to get a new bracket
    winningGames = [[i, j] for i in teams for j in rounds if resultsDF.loc[i, j] == 1]
    m.addConstr(quicksum(x[i, j] for [i, j] in winningGames) <= 62)

    # Optimize model with new constraint and record value
    m.optimize()
    objectiveScores.append(m.ObjVal)

    # Setup results from optimization in dataframe
    resultsx = []
    for v in m.getVars():
        #print("%s %g" % (v.varName, v.x))
        resultsx.append(v.x)
    reshapedresultsx = [resultsx[i:i+6] for i in range(0, 384, 6)]
    resultsDF = pd.DataFrame(reshapedresultsx, index = teams, columns=rounds)

    # Compute bracket score based on how it matches 2019 bracket
    bracketScore = 0
    for i in teams:
        k = 0
        for j in rounds:
            if resultsDF.loc[i, j] == origninalBracket.loc[i, j] and origninalBracket.loc[i,j] == 1:
                bracketScore = bracketScore + 2**k
            k+=1
    bracketScores.append(bracketScore)
    
# Print results
print(bracketScores)
print(objectiveScores)



Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 388 rows, 384 columns and 1684 nonzeros
Model fingerprint: 0xff8c53dc
Variable types: 0 continuous, 384 integer (384 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [4e-11, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 37.5775295
Presolve removed 48 rows and 48 columns
Presolve time: 0.00s
Presolved: 340 rows, 336 columns, 1620 nonzeros
Variable types: 0 continuous, 336 integer (336 binary)

Root relaxation: objective 9.567219e+01, 60 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   95.67219    0    2   37.57753   95.67219   155%     -    0s
H    0     0                      95.5566699   95.6721